# Demo SequenceActivity
The sequence activity calls all activities which are specified as sub processes.

In [2]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of the subprocesses used in the seqeunce activity
The sequence activity executes three basic activities in a sequence. Since the individual basic activities are executed one after the other, it is necessary to postpone the initialization of the initialization of these activities. This is done using the **postpone_start** parameter.

To ensure that all logging of the basic activities are available in a single log, additional logging is configured to an reporting activity.

In [3]:
reporting_activity_data = {
    "env": my_env,
    "name": "Reporting activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5k",  # For logging purposes
    "registry": registry,
    "duration": 0,
    "postpone_start": False,
}
reporting_activity = model.BasicActivity(**reporting_activity_data)

sub_processes = []
basic_activity_data1 = {
    "env": my_env,
    "name": "Basic activity1",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "duration": 14,
    "postpone_start": True,
    "additional_logs": [reporting_activity],
}
sub_processes.append(model.BasicActivity(**basic_activity_data1))
basic_activity_data2 = {
    "env": my_env,
    "name": "Basic activity2",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5c",  # For logging purposes
    "registry": registry,
    "duration": 5,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
}
sub_processes.append(model.BasicActivity(**basic_activity_data2))
basic_activity_data3 = {
    "env": my_env,
    "name": "Basic activity3",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "duration": 220,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
}
sub_processes.append(model.BasicActivity(**basic_activity_data3))



## Definition of seqeunce activity

In [4]:
sequential_activity_data = {
    "env": my_env,
    "name": "Sequential process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": sub_processes,
}
activity = model.SequentialActivity(**sequential_activity_data)

In [6]:
my_env.run()

In [7]:
log_df = pd.DataFrame(activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,sequential Sequential process,START,1970-01-01 00:00:00,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
1,sub process Basic activity1,START,1970-01-01 00:00:00,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
2,sub process Basic activity1,STOP,1970-01-01 00:00:14,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
3,sub process Basic activity2,START,1970-01-01 00:00:14,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
4,sub process Basic activity2,STOP,1970-01-01 00:00:19,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
5,sub process Basic activity3,START,1970-01-01 00:00:19,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
6,sub process Basic activity3,STOP,1970-01-01 00:03:59,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60
7,sequential Sequential process,STOP,1970-01-01 00:03:59,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff60


## Additional logging
The reporting activity log is displayed as well as the logs for the three basic activity sub processes are explicated and represented in an array.

In [8]:
rep_log_df = pd.DataFrame(reporting_activity.log)
data_rep = rep_log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data_rep = data_rep.drop_duplicates()
data_rep

,Message,ActivityState,Timestamp,Value,ActivityID
0,Reporting activity,START,1970-01-01 00:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
1,Basic activity1,START,1970-01-01 00:00:00,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
2,Reporting activity,STOP,1970-01-01 00:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
3,Basic activity1,STOP,1970-01-01 00:00:14,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
4,Basic activity2,START,1970-01-01 00:00:14,5,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
5,Basic activity2,STOP,1970-01-01 00:00:19,5,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
6,Basic activity3,START,1970-01-01 00:00:19,220,6dbbbdf7-4589-11e9-bf3b-b469212bff5d
7,Basic activity3,STOP,1970-01-01 00:03:59,220,6dbbbdf7-4589-11e9-bf3b-b469212bff5d


Logging of the individual basic activities

In [16]:
basic = pd.DataFrame()

for proc in sub_processes:
    basic = basic.append(pd.DataFrame(proc.log))
basic

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,Basic activity1,1970-01-01 00:00:00,14,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5b,START
1,Basic activity1,1970-01-01 00:00:14,14,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5b,STOP
0,Basic activity2,1970-01-01 00:00:14,5,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5c,START
1,Basic activity2,1970-01-01 00:00:19,5,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5c,STOP
0,Basic activity3,1970-01-01 00:00:19,220,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5d,START
1,Basic activity3,1970-01-01 00:03:59,220,None,6dbbbdf7-4589-11e9-bf3b-b469212bff5d,STOP
